# 1. Setup Mario

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py==8.2.1

In [ ]:
# Game
import gym_super_mario_bros
# Joypad
from nes_py.wrappers import JoypadSpace
# Controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement variations (action_space)
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Flag
done = True

# Loop through each frame
for step in range(100):
    if done:
        # Start the game
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
env.close()

# 2. Preprocess Environment

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 -f https://download.pytorch.org/whl/torch_stable.html
# PPO for RL
!pip install stable-baselines3[extra]==1.8.0

In [ ]:
# Frame Stacker Wrapper and Grayscaling Wrapper
# Framestack: Capture last couple of frames while Mario is playing
# GrayscaleObservation: Shave our data since it does not deal with no coloring, making our model faster
from gym.wrappers import GrayScaleObservation
# Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Plot the impact of framestacking
from matplotlib import pyplot as plt

In [ ]:
# Create the base env 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale the env
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside the dummy env
env = DummyVecEnv([lambda: env])
# Stack the frames
# Find how many frames to take in as you desire that works best
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# state = env.reset()
# plt.imshow(state)

In [ ]:
env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
#plt.figure(figsize=(20,16))
#for idx in range(state.shape[3]):
 #   plt.subplot(1,4,idx+1)
 #   plt.imshow(state[0][:,:,idx])
#plt.show()

# 3. Train the RL Model

In [ ]:
# AREA: Agent, Reward, Environment, Action
# PPO: Proximal Policy Optimization
# OS for file path management
import os
# PPO classifier
from stable_baselines3 import PPO
# Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# Save our model every x number of steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=500000, save_path=CHECKPOINT_DIR)

In [ ]:
# RL requires a policy network (such as the CNN that is the brain of the AI)
# CnnPolicy is fast at processing images, which fits our use case
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Train the model
# Every single game gets a million frames
#model.learn(total_timesteps=4000000, callback=callback)

In [ ]:
# No callback: model.save('name_of_file')
# To view the logs on tensorboard
# Activate the env on this dir and go into log dir (go into most recent ppo model)
#tensorboard --logdir=.

# 4. Test the Model

In [ ]:
# Load the model
model = PPO.load('./train/best_model_1500000', env=env)

In [ ]:
# Continue training the model
model.learn(total_timesteps=10000000, callback=callback)

In [ ]:
state = env.reset()

while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()